# Training a SVM model on TF-IDF vectorization of collected news articles of multiple classes
# Data collected using this tool https://github.com/prgmlu/youm-7-scraping

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import svm

# Loading the data, doing the train/test split with sklearn, and vectorizing the articles with sklearn TF-IDF implementation

In [24]:
data=pd.read_csv('data.csv')

articles=np.array(data['article'])
labels=np.array(data['label'])
x_train,x_test,y_train,y_test=train_test_split(articles,labels,test_size=0.2)

tf_vectorizer=TfidfVectorizer(min_df=4,max_df=0.3,ngram_range=(1,3))
x_train_tfidf=tf_vectorizer.fit_transform(x_train)
x_test_tfidf=tf_vectorizer.transform(x_test)

In [86]:
data.head()

,article,category,label,link,length
0,كرمت الدكتورة إيناس عبد الدايم وزيرة الثقافة و...,فن,1,https://www.youm7.com/story/2018/9/8/%D8%AA%D9...,437
1,يتبوأ النجوم الآسيويون منصات التفوق فى السينما...,فن,1,https://www.youm7.com/story/2018/9/8/Crazy-Ric...,1174
2,بعد نجاحه المتواصل على مدار أربع مواسم منذ عرض...,فن,1,https://www.youm7.com/story/2018/9/8/%D8%A8%D8...,688
3,شهد الحفل الغنائى الذى أحياه الفنان محمد شاهين...,فن,1,https://www.youm7.com/story/2018/9/8/%D8%A8%D9...,218
4,أكدت الفنانة ليلى عز العرب لـاليوم السابع أنها...,فن,1,https://www.youm7.com/story/2018/9/8/%D9%84%D9...,516


# Training a SVM model with the default parameters on the TF-IDF vectorization of the train data

In [25]:
clf = svm.LinearSVC()
clf.fit(x_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

# Testing the model on the TF-IDF vectorization of the test dataset

In [26]:
clf.score(x_test_tfidf,y_test)

0.9640044994375703

# Accuracy achieved on the test set is ~96%

# Making a function that takes an article and classifies it

In [80]:
def classify(article_text):
    global clf
    global tf_vectorizer
    global data
    article=tf_vectorizer.transform([article_text])
    labels=data[['label','category']].drop_duplicates()['label'].values
    categories=data[['label','category']].drop_duplicates()['category'].values
    look_up=dict(zip(labels,categories))
    return look_up[clf.predict(article)[0]]

# Using the above function to classify this <a href="https://www.youm7.com/story/2018/9/13/%D8%A7%D9%84%D9%86%D8%A7%D8%A6%D8%A8-%D8%B3%D8%B9%D8%AF-%D8%AA%D9%85%D8%B1%D8%A7%D8%B2-%D9%8A%D8%B7%D8%A7%D9%84%D8%A8-%D8%A7%D9%84%D8%AD%D9%83%D9%88%D9%85%D8%A9-%D8%A8%D8%AD%D9%84-%D8%A3%D8%B2%D9%85%D8%A9-%D9%85%D9%8A%D8%A7%D9%87-%D8%A7%D9%84%D8%B4%D8%B1%D8%A8-%D8%A8%D8%A7%D9%84%D8%A8%D8%AD%D9%8A%D8%B1%D8%A9/3948669">article</a>

In [84]:
artice_text="تقدم النائب محمد سعد تمراز، عضو مجلس النواب بمحافظة البحيرة، بسؤال للحكومة بشأن استمرار أزمة مياه الشرب بالمحافظة لعدد من القرى والتوابع.وأوضح تمراز لـاليوم السابع، أن هذه الأزمة ليست وليدة اللحظة، ولكنها منذ سنوات طويلة، وسبق وأن تم تقديم أكثر من طلب إحاطة وبيان عاجل وطلب مناقشة عامة خلال أدوار الانعقاد الثلاثة الماضية ولكن لم يتم وضع حلول جذرية لهذه الأزمة، التى أصبحت تشكل عبء إضافى على المواطن.وطالب عضو مجلس النواب بمحافظة البحيرة، الحكومة بوضع خطة للقضاء على هذه الأزمة من خلال جدول زمنى على أن يتم مراعاة القرى الأشد احتياجا، لافتًا إلى أن هناك عدد من القرى بها مشاريع مياه وصرف صحى متوقف العمل فيها على نسبة 10 أو 20% بحد أقصى، وعلى الوزارة أن تنتهى من هذه المشروعات ودخولها الخدمة لسرعة تخفيف الأعباء عن المواطنين لتوفير كوب مياه نظيف"

In [85]:
classify(artice_text)

'سياسة'

# The article is correctly classified.